In [1]:
import tweepy
from textblob import TextBlob
from wordcloud import WordCloud
import pandas as pd
import numpy as np
import re
import matplotlib.pyplot as plt
import time
from tqdm import tqdm_notebook as tqdm
import json

In [2]:
consumerKey = 'rpuYCEeUyzjGQ2hvC0JEQEmnf'
consumerSecret = 'XBoNeSo2bWRp1msgIb5NO8UYHOZuJETz9pNB4HDPRPlYScBH1y'
accessToken = '219520986-3KBJ4KRKR06s0q3yymyHpQKZx0LeTSpX7gpohSEp'
accessTokenSecret = 'ArvbcYIMvo6MXCl3KWrxw0IEdBuXaywakpSQGIOFYTbPC'

In [3]:
# Create the authentication object
authenticate = tweepy.OAuthHandler(consumerKey, consumerSecret) 
    
# Set the access token and access token secret
authenticate.set_access_token(accessToken, accessTokenSecret) 
    
# Creating the API object while passing in auth information
api = tweepy.API(authenticate, wait_on_rate_limit = True)

In [4]:
# Create a function to get the subjectivity
def getSubjectivity(text):
   return TextBlob(text).sentiment.subjectivity

# Create a function to get the polarity
def getPolarity(text):
   return  TextBlob(text).sentiment.polarity

In [46]:
us_state_abbrev = {
    'Alabama': 'AL',
    'Alaska': 'AK',
    'Arizona': 'AZ',
    'Arkansas': 'AR',
    'California': 'CA',
    'Colorado': 'CO',
    'Connecticut': 'CT',
    'Delaware': 'DE',
    'District of Columbia': 'DC',
    'Florida': 'FL',
    'Georgia': 'GA',
    'Hawaii': 'HI',
    'Idaho': 'ID',
    'Illinois': 'IL',
    'Indiana': 'IN',
    'Iowa': 'IA',
    'Kansas': 'KS',
    'Kentucky': 'KY',
    'Louisiana': 'LA',
    'Maine': 'ME',
    'Maryland': 'MD',
    'Massachusetts': 'MA',
    'Michigan': 'MI',
    'Minnesota': 'MN',
    'Mississippi': 'MS',
    'Missouri': 'MO',
    'Montana': 'MT',
    'Nebraska': 'NE',
    'Nevada': 'NV',
    'New Hampshire': 'NH',
    'New Jersey': 'NJ',
    'New Mexico': 'NM',
    'New York': 'NY',
    'North Carolina': 'NC',
    'North Dakota': 'ND',
    'Ohio': 'OH',
    'Oklahoma': 'OK',
    'Oregon': 'OR',
    'Pennsylvania': 'PA',
    'Rhode Island': 'RI',
    'South Carolina': 'SC',
    'South Dakota': 'SD',
    'Tennessee': 'TN',
    'Texas': 'TX',
    'Utah': 'UT',
    'Vermont': 'VT',
    'Virginia': 'VA',
    'Washington': 'WA',
    'West Virginia': 'WV',
    'Wisconsin': 'WI',
    'Wyoming': 'WY'
}

In [47]:
states = {
    'AL': {'subj': [], 'pol': []},
    'AK': {'subj': [], 'pol': []},
    'AZ': {'subj': [], 'pol': []},
    'AR': {'subj': [], 'pol': []},
    'CA': {'subj': [], 'pol': []},
    'CO': {'subj': [], 'pol': []},
    'CT': {'subj': [], 'pol': []},
    'DE': {'subj': [], 'pol': []},
    'DC': {'subj': [], 'pol': []},
    'FL': {'subj': [], 'pol': []},
    'GA': {'subj': [], 'pol': []},
    'HI': {'subj': [], 'pol': []},
    'ID': {'subj': [], 'pol': []},
    'IL': {'subj': [], 'pol': []},
    'IN': {'subj': [], 'pol': []},
    'IA': {'subj': [], 'pol': []},
    'KS': {'subj': [], 'pol': []},
    'KY': {'subj': [], 'pol': []},
    'LA': {'subj': [], 'pol': []},
    'ME': {'subj': [], 'pol': []},
    'MD': {'subj': [], 'pol': []},
    'MA': {'subj': [], 'pol': []},
    'MI': {'subj': [], 'pol': []},
    'MN': {'subj': [], 'pol': []},
    'MS': {'subj': [], 'pol': []},
    'MO': {'subj': [], 'pol': []},
    'MT': {'subj': [], 'pol': []},
    'NE': {'subj': [], 'pol': []},
    'NV': {'subj': [], 'pol': []},
    'NH': {'subj': [], 'pol': []},
    'NJ': {'subj': [], 'pol': []},
    'NM': {'subj': [], 'pol': []},
    'NY': {'subj': [], 'pol': []},
    'NC': {'subj': [], 'pol': []},
    'ND': {'subj': [], 'pol': []},
    'OH': {'subj': [], 'pol': []},
    'OK': {'subj': [], 'pol': []},
    'OR': {'subj': [], 'pol': []},
    'PA': {'subj': [], 'pol': []},
    'RI': {'subj': [], 'pol': []},
    'SC': {'subj': [], 'pol': []},
    'SD': {'subj': [], 'pol': []},
    'TN': {'subj': [], 'pol': []},
    'TX': {'subj': [], 'pol': []},
    'UT': {'subj': [], 'pol': []},
    'VT': {'subj': [], 'pol': []},
    'VA': {'subj': [], 'pol': []},
    'WA': {'subj': [], 'pol': []},
    'WV': {'subj': [], 'pol': []},
    'WI': {'subj': [], 'pol': []},
    'WY': {'subj': [], 'pol': []}
}

In [25]:
with open('0322.jsonl', 'r', encoding='utf-8') as json_file:
    json_list = list(json_file)

pbar = tqdm(total=len(json_list))
for json_str in json_list:
    tweet = json.loads(json_str)
#     print(tweet['place'])
    place = tweet['place']
    if place and place['country_code']=="US":
        fullName = place['full_name'].split(', ')
        state = fullName[-1]
        if state == 'USA':
            state = fullName[0]
        if state in us_state_abbrev:
            state = us_state_abbrev[state]
        if state in states:
            text = tweet['full_text']
            subj = getSubjectivity(text)
            pol = getPolarity(text)
#             if subj != 0 and pol != 0:
            states[state]['subj'].append(subj)
            states[state]['pol'].append(pol)
    pbar.update(1)
pbar.close()

100%|█████████████████████████████████████████████████████████████████████████| 73781/73781 [00:04<00:00, 16365.07it/s]


In [26]:
senti = {}
for state in states:
    subj = np.mean(states[state]['subj'])
    pol = np.mean(states[state]['pol'])
    senti[state] = [pol, subj, len(states[state]['pol']), len(states[state]['subj'])]
senti

C:\Users\alvin\Anaconda3\lib\site-packages\numpy\core\fromnumeric.py:2920: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
C:\Users\alvin\Anaconda3\lib\site-packages\numpy\core\_methods.py:85: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


{'AL': [0.21332958491161616, 0.3651515151515151, 9, 9],
 'AK': [0.06818181818181818, 0.22727272727272727, 2, 2],
 'AZ': [0.026692057942057947, 0.32205086580086584, 26, 26],
 'AR': [0.0029629629629629563, 0.2925925925925926, 5, 5],
 'CA': [0.06523082781612757, 0.31850962435146113, 196, 196],
 'CO': [-0.006287202380952376, 0.35751426091269844, 16, 16],
 'CT': [-0.19090909090909092, 0.43863636363636366, 4, 4],
 'DE': [0.03933333333333334, 0.24511111111111109, 2, 2],
 'DC': [-0.09979166666666667, 0.27875, 8, 8],
 'FL': [0.07523473148473148, 0.2383285070785071, 37, 37],
 'GA': [0.001613965744400524, 0.1152832674571805, 23, 23],
 'HI': [0.2633375420875421, 0.37942760942760945, 12, 12],
 'ID': [0.0675, 0.22833333333333333, 5, 5],
 'IL': [0.05506333172999839, 0.35021164021164025, 27, 27],
 'IN': [0.13290719696969697, 0.358822601010101, 16, 16],
 'IA': [0.05109126984126985, 0.2726190476190476, 3, 3],
 'KS': [0.15555555555555556, 0.2885416666666667, 6, 6],
 'KY': [0.16666666666666669, 0.21666666

In [29]:
hello = pd.read_csv('Dailies/2020-03-22_clean-dataset.tsv', delimiter='\t')

In [32]:
tweetsBy100 = []
i = 0
tempTweets = []
for tweetId in hello['tweet_id']:
    tempTweets.append(tweetId)
    if i == 99:
        tweetsBy100.append(tempTweets)
        tempTweets = []
        i = -1
    i += 1
if len(tempTweets) > 0:
    tweetsBy100.append(tempTweets)

In [48]:
pbar = tqdm(total=len(tweetsBy100))
for batch in tweetsBy100:
    tweets = api.statuses_lookup(batch)
    pbar.update(1)
    for tweet in tweets:
        place = tweet.place
        if place and place.country_code=="US":
            fullName = place.full_name.split(', ')
            state = fullName[-1]
            if state == 'USA':
                state = fullName[0]
            if state in us_state_abbrev:
                state = us_state_abbrev[state]
            if state in states:
                text = tweet.text
                subj = getSubjectivity(text)
                pol = getPolarity(text)
                states[state]['subj'].append(subj)
                states[state]['pol'].append(pol)
pbar.close()

 13%|██████████▏                                                                 | 1043/7797 [12:40<1:30:55,  1.24it/s]

KeyboardInterrupt: 

In [50]:
senti = {}
for state in states:
    subj = np.mean(states[state]['subj'])
    pol = np.mean(states[state]['pol'])
    senti[state] = [pol, subj, len(states[state]['pol']), len(states[state]['subj'])]
senti

C:\Users\alvin\Anaconda3\lib\site-packages\numpy\core\fromnumeric.py:2920: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
C:\Users\alvin\Anaconda3\lib\site-packages\numpy\core\_methods.py:85: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


{'AL': [0.19444444444444445, 0.3888888888888889, 9, 9],
 'AK': [0.06818181818181818, 0.22727272727272727, 2, 2],
 'AZ': [0.05193803110469778, 0.34932058682058686, 27, 27],
 'AR': [0.009999999999999998, 0.3, 5, 5],
 'CA': [0.07162345671955414, 0.2987471010123593, 213, 213],
 'CO': [-0.017853785874619197, 0.3981260020843354, 18, 18],
 'CT': [-0.17272727272727273, 0.2909090909090909, 5, 5],
 'DE': [0.0775, 0.30999999999999994, 2, 2],
 'DC': [-0.103125, 0.22916666666666666, 8, 8],
 'FL': [0.062266855484940604, 0.3029698581560284, 47, 47],
 'GA': [0.001613965744400524, 0.1152832674571805, 23, 23],
 'HI': [0.22398989898989896, 0.3176515151515152, 15, 15],
 'ID': [0.0675, 0.22833333333333333, 5, 5],
 'IL': [0.06407312925170067, 0.3171981292517007, 28, 28],
 'IN': [0.14801136363636364, 0.361947601010101, 16, 16],
 'IA': [0.13214285714285715, 0.2849206349206349, 3, 3],
 'KS': [0.15138888888888888, 0.2885416666666667, 6, 6],
 'KY': [0.18703703703703706, 0.2314814814814815, 9, 9],
 'LA': [0.02545

In [12]:
stateBatches = {
#     'states1' : ['AK', 'AL', 'AR', "AZ", 'CA'],
#     'states2' : ["CO", 'CT', 'DC', 'DE', 'FL'],
#     'states3' : ['GA', 'HI', 'IA', 'ID', 'IL'],
#     'states4' : ['IN', 'KS', 'KY', 'LA', 'MA'],
    'states5' : ['MD', 'ME', "MI", 'MN', "MO"],
    'states6' : ["MS", 'MT', 'NC', 'ND', 'NE'],
    'states7' : ['NH', 'NJ', 'NM', 'NV', 'NY'],
    'states8' : ['OH', 'OK', 'OR', 'PA', 'RI'],
    'states9' : ['SC', 'SD', 'TN', 'TX', 'UT'],
    'states10' : ['VA', 'VT', 'WA', 'WI', 'WV', 'WY']
}

In [6]:
def readCSV(states):
    ids = {}
    batchNum = 0
    for state in states:
        stateTweets = pd.read_csv('Dailies/'+state+'.csv')['Tweet_ID']
        i = 0
        tweetIds = []
        tempTweets = []
        for tweetId in stateTweets:
            tempTweets.append(tweetId)
            if i == 99:
                tweetIds.append(tempTweets)
                tempTweets = []
                i = -1
                batchNum += 1
            i += 1
        if len(tempTweets) > 0:
            tweetIds.append(tempTweets)
            batchNum += 1
        ids[state] = tweetIds
    return ids, batchNum

In [7]:
def makeCSV(ids, batchNum):
    info = []
    pbar = tqdm(total=batchNum)
    for state in ids.keys():
        for batch in ids[state]:
            pbar.update(1)
            tweets = api.statuses_lookup(batch)
            for tweet in tweets:
                text = tweet.text
                date = str(tweet.created_at).split(" ")[0]
                subj = getSubjectivity(text)
                pol = getPolarity(text)
                info.append([state, date, tweet.id, subj, pol])
    return info
    pbar.close()

In [8]:
def saveCSV(info, name):
    saver = pd.DataFrame(info)
    saver.columns = ['state', 'date', 'tweet_id', 'subj', 'pol']
    saver.to_csv(name+".csv", index=False)

In [13]:
for stateBatch in stateBatches.keys():
    test, batchNum = readCSV(stateBatches[stateBatch])
    test2 = makeCSV(test, batchNum)
    saveCSV(test2, stateBatch)

In [26]:
hello = pd.read_csv('Dailies/NY.csv', delimiter='\t')

In [27]:
tweetsBy100 = []
i = 0
tempTweets = []
for tweetId in hello['Tweet_ID']:
    tempTweets.append(tweetId)
    if i == 99:
        tweetsBy100.append(tempTweets)
        tempTweets = []
        i = -1
    i += 1
if len(tempTweets) > 0:
    tweetsBy100.append(tempTweets)

In [28]:
ny = {}
info = []

In [29]:
pbar = tqdm(total=len(tweetsBy100))
for batch in tweetsBy100:
    tweets = api.statuses_lookup(batch)
    pbar.update(1)
    for tweet in tweets:
        text = tweet.text
        date = str(tweet.created_at).split(" ")[0]
        subj = getSubjectivity(text)
        pol = getPolarity(text)
        if subj != 0 or pol != 0:
            info.append(['NY', date, tweet.id, subj, pol])
pbar.close()






  0%|                                                                                          | 0/581 [00:00<?, ?it/s]




  0%|▏                                                                                 | 1/581 [00:00<07:46,  1.24it/s]




  0%|▎                                                                                 | 2/581 [00:01<08:51,  1.09it/s]




  1%|▍                                                                                 | 3/581 [00:03<09:35,  1.00it/s]




  1%|▌                                                                                 | 4/581 [00:04<09:27,  1.02it/s]




  1%|▋                                                                                 | 5/581 [00:05<10:02,  1.05s/it]




  1%|▊                                                                                 | 6/581 [00:06<09:47,  1.02s/it]




  1%|▉                                                                                 | 7/581 [00:07<09:40,  1.01s/it]




  1

 23%|██████████████████▏                                                             | 132/581 [02:09<07:32,  1.01s/it]




 23%|██████████████████▎                                                             | 133/581 [02:10<07:51,  1.05s/it]




 23%|██████████████████▍                                                             | 134/581 [02:11<07:37,  1.02s/it]




 23%|██████████████████▌                                                             | 135/581 [02:12<07:34,  1.02s/it]




 23%|██████████████████▋                                                             | 136/581 [02:13<07:19,  1.01it/s]




 24%|██████████████████▊                                                             | 137/581 [02:14<07:16,  1.02it/s]




 24%|███████████████████                                                             | 138/581 [02:16<08:45,  1.19s/it]




 24%|███████████████████▏                                                            | 139/581 [02:17<08:26,  1.15s/it]




 24%|███

 45%|████████████████████████████████████▎                                           | 264/581 [04:22<05:17,  1.00s/it]




 46%|████████████████████████████████████▍                                           | 265/581 [04:23<05:11,  1.01it/s]




 46%|████████████████████████████████████▋                                           | 266/581 [04:24<04:53,  1.07it/s]




 46%|████████████████████████████████████▊                                           | 267/581 [04:25<05:08,  1.02it/s]




 46%|████████████████████████████████████▉                                           | 268/581 [04:26<05:04,  1.03it/s]




 46%|█████████████████████████████████████                                           | 269/581 [04:27<05:07,  1.01it/s]




 46%|█████████████████████████████████████▏                                          | 270/581 [04:28<05:08,  1.01it/s]




 47%|█████████████████████████████████████▎                                          | 271/581 [04:29<05:23,  1.04s/it]




 47%|███

 68%|██████████████████████████████████████████████████████▌                         | 396/581 [06:47<03:28,  1.13s/it]




 68%|██████████████████████████████████████████████████████▋                         | 397/581 [06:49<03:30,  1.15s/it]




 69%|██████████████████████████████████████████████████████▊                         | 398/581 [06:50<03:45,  1.23s/it]




 69%|██████████████████████████████████████████████████████▉                         | 399/581 [06:51<03:45,  1.24s/it]




 69%|███████████████████████████████████████████████████████                         | 400/581 [06:52<03:33,  1.18s/it]




 69%|███████████████████████████████████████████████████████▏                        | 401/581 [06:53<03:22,  1.13s/it]




 69%|███████████████████████████████████████████████████████▎                        | 402/581 [06:54<03:21,  1.13s/it]




 69%|███████████████████████████████████████████████████████▍                        | 403/581 [06:56<03:21,  1.13s/it]




 70%|███

 91%|████████████████████████████████████████████████████████████████████████▋       | 528/581 [09:11<00:53,  1.00s/it]




 91%|████████████████████████████████████████████████████████████████████████▊       | 529/581 [09:12<00:49,  1.05it/s]




 91%|████████████████████████████████████████████████████████████████████████▉       | 530/581 [09:13<00:52,  1.03s/it]




 91%|█████████████████████████████████████████████████████████████████████████       | 531/581 [09:14<00:52,  1.05s/it]




 92%|█████████████████████████████████████████████████████████████████████████▎      | 532/581 [09:15<00:51,  1.05s/it]




 92%|█████████████████████████████████████████████████████████████████████████▍      | 533/581 [09:16<00:48,  1.01s/it]




 92%|█████████████████████████████████████████████████████████████████████████▌      | 534/581 [09:17<00:44,  1.07it/s]




 92%|█████████████████████████████████████████████████████████████████████████▋      | 535/581 [09:18<00:42,  1.07it/s]




 92%|███

In [30]:
ny

{'2020-03-29': {'subj': [0.4,
   0.45,
   0.75,
   0.6333333333333333,
   0.1,
   0.5,
   1.0,
   0.8,
   0.5678571428571428,
   0.5,
   0.75,
   0.4,
   0.6000000000000001,
   0.5,
   0.32222222222222224,
   0.6,
   0.05,
   0.1,
   0.3011904761904762,
   0.9,
   0.7,
   0.8,
   0.75,
   0.06666666666666667,
   0.6,
   0.32785714285714285,
   0.06666666666666667,
   0.37486772486772485,
   0.5833333333333334,
   0.525,
   0.43333333333333335,
   0.4,
   0.4,
   0.3333333333333333,
   0.3,
   0.6400000000000001,
   0.6,
   0.36666666666666664,
   0.4375,
   1.0,
   0.5,
   0.3462962962962963,
   0.4,
   1.0,
   0.5333333333333333,
   1.0,
   0.5727272727272728,
   0.7,
   1.0,
   0.9444444444444444,
   0.16666666666666666,
   0.3,
   0.45,
   0.8,
   0.1,
   0.625,
   1.0,
   0.35555555555555557,
   0.7833333333333333,
   0.6375,
   0.5,
   0.95,
   0.3,
   0.06666666666666667,
   0.35000000000000003,
   0.7000000000000001,
   0.6,
   0.4,
   0.5454545454545454,
   0.275,
   0.225,
   